In [7]:
import warnings 
warnings.simplefilter('ignore')

import scanpy as sc
import anndata
import scparadise
import pandas as pd
import muon as mu
import numpy as np
import pandas as pd
import os
import scvi
import torch

In [2]:
os.chdir('/mnt/c/Users/vadim/scRNA/scParadise/scripts_article')

In [9]:
# Load normalized integrated data
mdata = mu.read_h5mu('PBMC_3p_CITE/human_PBMC_mdata_unintegrated.h5mu')

In [10]:
# Subset genes
adata = mdata.mod['rna'].copy()
del mdata
adata = anndata.AnnData(X = adata.layers['counts'],
                        obs = adata.obs,
                        var = adata.var,
                        obsm = adata.obsm) 
# Create list of samples to leave in reference train dataset
lst_reference_1 = ['P8_0', 'P7_0', 'P2_0', 'P6_0']
# Create list of samples to leave in reference train dataset
lst_reference_2 = ['P1_0', 'P5_0', 'P3_0', 'P4_0']
# Create adata_train - 8 samples of 8 donors from unintegrated adata object (8 donors, 24 samples)
adata_train_1 = adata[adata.obs['orig.ident'].isin(lst_reference_1)].copy()
adata_train_2 = adata[adata.obs['orig.ident'].isin(lst_reference_2)].copy()
# Create adata_train1, adata_train2 and adata_test datasets
adata_test_1 = adata[adata.obs['orig.ident'].isin(['P1_3', 'P3_3'])].copy()
adata_test_2 = adata[adata.obs['orig.ident'].isin(['P1_7', 'P8_3'])].copy()
adata_test_3 = adata[adata.obs['orig.ident'].isin(['P2_3', 'P4_7'])].copy()
adata_test_4 = adata[adata.obs['orig.ident'].isin(['P2_7', 'P6_3'])].copy()
adata_test_5 = adata[adata.obs['orig.ident'].isin(['P3_7', 'P7_3'])].copy()
adata_test_6 = adata[adata.obs['orig.ident'].isin(['P4_3', 'P7_7'])].copy()
adata_test_7 = adata[adata.obs['orig.ident'].isin(['P5_3', 'P8_7'])].copy()
adata_test_8 = adata[adata.obs['orig.ident'].isin(['P5_7', 'P6_7'])].copy()

# Normalize data, find highly variable features
for i in [adata_train_1, adata_train_2, adata_test_1, adata_test_2, adata_test_3, adata_test_4, adata_test_5, adata_test_6, adata_test_7, adata_test_8]:
    i.layers['counts'] = i.X.copy()
    sc.pp.normalize_total(i, target_sum=None)
    sc.pp.log1p(i)
    i.raw = i

# Subset anndata object based on a selected marker genes
genes = pd.read_csv('PBMC_3p_CITE/genes_for_AI.csv')
adata_train_1 = adata_train_1[:, genes.genes].copy()

In [15]:
# Create lists with paired samples and annotation levels
lst_test_folders = ['P1_3_P3_3', 'P1_7_P8_3', 'P2_3_P4_7', 'P2_7_P6_3', 'P3_7_P7_3', 'P4_3_P7_7', 'P5_3_P8_7', 'P5_7_P6_7']
lst_test_adatas = [adata_test_1, adata_test_2, adata_test_3, adata_test_4, adata_test_5, adata_test_6, adata_test_7, adata_test_8]
lst_annotations = ['celltype_l3', 'celltype_l2', 'celltype_l1']

In [14]:
# Train scvi model for adata_train dataset
scvi.model.SCVI.setup_anndata(adata_train_1, batch_key="orig.ident", layer="counts")
scvi_ref = scvi.model.SCVI(adata_train_1)
scvi_ref.train()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 269/269: 100%|██████████████| 269/269 [18:54<00:00,  4.71s/it, v_num=1, train_loss_step=383, train_loss_epoch=375]

`Trainer.fit` stopped: `max_epochs=269` reached.


Epoch 269/269: 100%|██████████████| 269/269 [18:54<00:00,  4.22s/it, v_num=1, train_loss_step=383, train_loss_epoch=375]


In [16]:
scvi_ref_path = 'PBMC_3p_CITE/scanvi_test/models'
scvi_ref.save(scvi_ref_path, overwrite=True)

In [17]:
for folder in lst_test_folders:
    os.makedirs(os.path.join('PBMC_3p_CITE/scanvi_test/reports', folder))
for i in lst_annotations:
    os.makedirs(os.path.join('PBMC_3p_CITE/scanvi_test/models', i))

In [18]:
for i in lst_annotations:
    # Train scanvi model for reference dataset (adata_train)
    scanvi_ref = scvi.model.SCANVI.from_scvi_model(
        scvi_ref,
        unlabeled_category="Unknown",
        labels_key=i,
    )
    # Train scanvi models for selected annotation level
    scanvi_ref.train()
    scanvi_ref_path = os.path.join('PBMC_3p_CITE/scanvi_test/models', i)
    scanvi_ref.save(scanvi_ref_path, overwrite=True)
    
    for adata_test, folder in zip(lst_test_adatas, lst_test_folders):
        # Train scanvi model for test dataset (adata_test)
        scvi.model.SCANVI.prepare_query_anndata(adata_test, scanvi_ref)
        scanvi_test = scvi.model.SCANVI.load_query_data(adata_test, scanvi_ref)
        scanvi_test.train(
            max_epochs=10
        )
        # Create name for predicted annotation level
        SCANVI_annotation = 'scanvi_' + i
        # Predict annotation using pretrained scanvi_test model
        adata_test.obs[SCANVI_annotation] = scanvi_test.predict()
        # Create and save classification report of selected annotation level
        file_save = 'report_test_scanvi_default_' + i + '.csv'
        scparadise.scnoah.report_classif_full(adata_test, 
                                              celltype = i, 
                                              pred_celltype = SCANVI_annotation, 
                                              report_name = file_save,
                                              save_path = os.path.join('PBMC_3p_CITE/scanvi_test/reports', folder),
                                              save_report=True)

INFO     Training for 10 epochs.                                                                                   


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 10/10: 100%|██████████████████| 10/10 [01:44<00:00, 10.58s/it, v_num=1, train_loss_step=375, train_loss_epoch=366]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 10/10: 100%|██████████████████| 10/10 [01:44<00:00, 10.44s/it, v_num=1, train_loss_step=375, train_loss_epoch=366]
INFO     Found 100.0% reference vars in query data.                                                                
INFO     Training for 10 epochs.                                                                                   


...

Epoch 10/10: 100%|██████████████████| 10/10 [00:34<00:00,  3.42s/it, v_num=1, train_loss_step=368, train_loss_epoch=381]
Successfully saved report

INFO     Found 100.0% reference vars in query data.                                                                
INFO     Training for 10 epochs.                                                                                   


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 10/10: 100%|██████████████████| 10/10 [00:33<00:00,  3.28s/it, v_num=1, train_loss_step=366, train_loss_epoch=368]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 10/10: 100%|██████████████████| 10/10 [00:33<00:00,  3.34s/it, v_num=1, train_loss_step=366, train_loss_epoch=368]
Successfully saved report

